In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

dataset_dir = 'data_base'

In [9]:
image_height = 300
image_width = 300

model = keras.Sequential([
    keras.layers.Flatten(input_shape = (image_height, image_width)),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(10)
])

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy'])

print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_3 (Flatten)             │ (None, 90000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    11,520,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,521,418 (43.95 MB)

 Trainable params: 11,521,418 (43.95 MB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
# separar dados de treino e teste, alem das labels

batch_size = 32
validation_split = 0.2 
test_split = 0.1

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(image_height, image_width),
    interpolation='nearest',
    batch_size=batch_size,
    shuffle=True,
    seed=123,
    validation_split=validation_split,
    subset='training'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(image_height, image_width),
    interpolation='nearest',
    batch_size=batch_size,
    shuffle=True,
    seed=123,
    validation_split=validation_split,
    subset='validation'
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Classes encontradas: {class_names}")
print(f"Número total de classes: {num_classes}")

Found 4670 files belonging to 26 classes.
Using 3736 files for training.
Found 4670 files belonging to 26 classes.
Using 934 files for validation.
Classes encontradas: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
Número total de classes: 26


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)), # normaliza pixels de 0-255 para 0-1
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes) 
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)         │ (None, 300, 300, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 298, 298, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 149, 149, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 147, 147, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 73, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 341056)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │    43,655,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 26)             │         3,354 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,678,042 (166.62 MB)

 Trainable params: 43,678,042 (166.62 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
epochs = 4
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/4
117/117 ━━━━━━━━━━━━━━━━━━━━ 236s 2s/step - accuracy: 1.0000 - loss: 1.3044e-05 - val_accuracy: 0.9979 - val_loss: 0.0156
Epoch 2/4
117/117 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 1.0000 - loss: 8.2324e-06 - val_accuracy: 0.9979 - val_loss: 0.0156
Epoch 3/4
117/117 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 1.0000 - loss: 6.6013e-06 - val_accuracy: 0.9979 - val_loss: 0.0156
Epoch 4/4
117/117 ━━━━━━━━━━━━━━━━━━━━ 271s 2s/step - accuracy: 1.0000 - loss: 5.5362e-06 - val_accuracy: 0.9979 - val_loss: 0.0156


In [14]:
model.save('models/model.h5')
model.save('models/model.keras')